In [1]:
import pandas as pd

df = pd.read_csv("../Data/Unprocess.csv")
# df의 "A"열이 '\x0c'와 다른 것만 저장
df = df[df["A"] != '\x0c']
df = df.dropna()
df = df.drop_duplicates()
df.to_csv("../Data/Unprocess.csv", index=False)
df.Category.value_counts()

# 열 내용 변경 : 1. アニメゲがゲーム -> アニメゲーム 로 변경
#               2. だっ -> 雑学으로 변경
#               3. レちで -> スポーツ 로 변경
df = pd.read_csv("../Data/Unprocess.csv")
df.loc[df["Category"] == " アニメゲがゲーム", "Category"] = "アニメゲーム"
df.loc[df["Category"] == "だっ", "Category"] = "雑学"
df.loc[df["Category"] == "レちで", "Category"] = "スポーツ"

df.to_csv("../Data/Unprocess.csv", index=False)
df.Category.value_counts()

Category
理系          3933
芸能          3576
雑学          3203
スポーツ        3063
文系          3063
アニメゲーム      2923
Category       1
Name: count, dtype: int64

In [2]:
# 해당 unique값을 바탕으로 6개의 DF생성 후 파일로 저장
for i in df.Category.unique():
    df_temp = df[df["Category"] == i]
    df_temp.to_csv(f"../Data/{i}.csv", index=False)

In [3]:
import pandas as pd
from Korpora import Korpora
from konlpy.tag import Okt
# 일본어 tokenize import
from janome.tokenizer import Tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from joblib import dump
from joblib import load

In [4]:
Genre = ['雑学', 'スポーツ', '理系', '芸能', 'アニメゲーム', '文系']

for i in Genre:
    df = pd.read_csv(f"../Data/{i}.csv", usecols =["Q","K1","K2","K3","K4","A"])

    # 각 열을 문자열로 합쳐서 튜플 리스트 생성
    QnA = [(q + ":" + k1 +","+ k2 +","+ k3 +","+ k4, index, a) for index, q, k1, k2, k3, k4, a in zip(df.index, df['Q'], df['K1'], df['K2'], df['K3'], df['K4'], df["A"])]

    data = [(Question, index) for Question, index, Answer in QnA]
    # 텍스트 벡터화와 분류 모델을 포함한 파이프라인 생성
    model = make_pipeline(
        TfidfVectorizer(),
        MultinomialNB()
    )
    # 모델 학습
    X_train = [item[0] for item in data]
    y_train = [item[1] for item in data]
    model.fit(X_train, y_train)
    # 모델 저장
    dump(model, f'../Model/sim_{i}.joblib')

In [5]:
from joblib import load
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pyautogui
from CRAWL import *
import pandas as pd
# 유사도 모델 로드 

Genre_list = ['雑学', 'スポーツ', '理系', '芸能', 'アニメゲーム', '文系']
# Genre 내용을 사용하여 모델 변수 6개를 만들고, 모델 불러오기
雑学= load('../Model/sim_雑学.joblib')
スポーツ = load('../Model/sim_スポーツ.joblib')
理系 = load('../Model/sim_理系.joblib')
芸能 = load('../Model/sim_芸能.joblib')
アニメゲーム = load('../Model/sim_アニメゲーム.joblib')
文系 = load('../Model/sim_文系.joblib')
Genre_model = [雑学, スポーツ, 理系, 芸能, アニメゲーム, 文系]

In [6]:
flag=0 # 데이터 추가 변수
# 화면 클릭 함수
def click_answer(most_similar_index):
    if most_similar_index==0 :
        pyautogui.click(950, 980)  
    elif most_similar_index==1:
        pyautogui.click(950, 1110) 
    elif most_similar_index==2:
        pyautogui.click(950, 1244)
    elif most_similar_index==3:
        pyautogui.click(950, 1363) 
        
def find_answer_location(imgpath):
    screenshot = np.array(pyautogui.screenshot())# 전체화면 캡처
    gray = cv2.cvtColor(screenshot, cv2.COLOR_BGR2GRAY) # O.png탐색
    result = cv2.matchTemplate(gray, cv2.imread(imgpath, 0), cv2.TM_CCOEFF_NORMED)# 이미지 검출
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)# 최대값 찾기
    return max_loc     # 최대값 위치 반환

# 정답 찾기
def find_answer():
    # 1초동안 실행하기
    for _ in range(5) :
        x,y = find_answer_location('../Img/XX.png')
        if 1050<=x<=1150 and 600<=y<=700:
            x,y = find_answer_location('../Img/O.png')
            if 940<=x<=1050 and 970<=y<=1094:
                return 1
            if 940<=x<=1050 and 1100<=y<=1210:
                return 2
            if 940<=x<=1050 and 1234<=y<=1341:
                return 3
            if 940<=x<=1050 and 1353<=y<=1484:
                return 4
        time.sleep(0.1)
    return 0

def levenshtein_distance(s1, s2):
    m, n = len(s1), len(s2)
    dp = [[0] * (n + 1) for _ in range(m + 1)]

    for i in range(m + 1):
        dp[i][0] = i
    for j in range(n + 1):
        dp[0][j] = j

    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if s1[i - 1] == s2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]
            else:
                dp[i][j] = min(dp[i - 1][j], dp[i][j - 1], dp[i - 1][j - 1]) + 1

    return dp[m][n]

In [9]:
mismatch = pd.read_csv('data.csv')
# 문제 맞추기 시작
while True:
    txt_list = capture_screen()
    if len(txt_list)<5:
        print("선지가 없습니다.")
        break
    Question, Genre, Nominee = extract(txt_list)
    if Genre=="だっ" : Genre="雑学"
    if Genre==" アニメゲがゲーム" : Genre="アニメゲーム"
    if Genre=="茎能" : Genre="芸能"
    model_index= Genre_list.index(Genre)
    model = Genre_model[model_index]
    df = pd.read_csv(f"../Data/{Genre}.csv",usecols =["Q","K1","K2","K3","K4","A"])
    # 각 열을 문자열로 합쳐서 튜플 리스트 생성
    QnA = [(q + ":" + k1 +","+ k2 +","+ k3 +","+ k4, index, a) for index, q, k1, k2, k3, k4, a in zip(df.index, df['Q'], df['K1'], df['K2'], df['K3'], df['K4'], df["A"])]

    # 입력 문장을 예측
    input_text =",".join((Question, Nominee[0], Nominee[1], Nominee[2], Nominee[3]))
    predicted_label = model.predict([input_text])[0]    
    print(f"{'='*50}\n1.{QnA[predicted_label][0]}")
    print(f"2.{Question}\n{'='*50}")
    
    # 예측된 레이블에 해당하는 질문 가져오기
    predicted_question = QnA[predicted_label][2]

    if QnA[predicted_label][2] in Nominee:
        most_similar_index = Nominee.index(QnA[predicted_label][2])
    else :
        # 문장 벡터화
        try:
            distances = [levenshtein_distance(predicted_question, text) for text in Nominee]
            # 가장 유사한 단어 추출
            most_similar_index = distances.index(min(distances))
            print("➡️Vectorization : ",[predicted_question] + [nominee for nominee in Nominee])

        except:
            most_similar_index = 0
    # 결과 출력 & 클릭
    print("정답 :", most_similar_index,"🤩")
    click_answer(most_similar_index)
    answer_num=find_answer()
    if answer_num : # 문제를 틀렸다면
        answer=Nominee[answer_num-1]
        mismatch.loc[len(df)] = [Question, Genre, Nominee[0], Nominee[1], Nominee[2], Nominee[3], answer]
        mismatch.to_csv('data.csv', index=False)
    time.sleep(1.5)

1.日本テレビのバラエティ番組『なんでもワールドランキング ネプ&イモトの〇の〇O〇』 。 〇の〇の〇は何 ?:世界階級,世界ランク,世界序列,世界番付
2.バラエティ番組『なんでもワールドランキング ネプ&〇〇〇の世界番付』。〇〇〇に入る名前は何 ?
➡️Vectorization :  ['世界番付', 'イモト', 'デガワ', 'アリヨシ', 'ローフラ']
정답 : 0 🤩
1.演劇用語で、長期にわたって同じ演目を上演することを何という ?:ミドルラン,セカンドラン,ショートラン,ロングラン
2.演劇用語で、長期にわたって同じ演目を上演することを何という ?
정답 : 3 🤩
1.陸上競技やテストなどで時間をはかるための道具は「〇〇〇ウォッナ」。〇〇〇は何 ?:スキップ,ステップ,ストップ,スリップ
2.定力の弱い人が目に装着する「C〇C)〇っレンズ」。〇〇〇は何 2
➡️Vectorization :  ['スリップ', 'コンバパイル', 'コンタクト', 'コンサート', 'コンバート']
정답 : 1 🤩
1.陸奥湾を隔てて下北人邊島に対財する、青森県の生島はどこ2:津軽生島,紀伊生島,能登生島,房総生島
2.次のうち、日本海側にある生島はどれ 2?
➡️Vectorization :  ['津軽生島', '能登生島', '紀伊年島', '房総生島', '大隈生島']
정답 : 0 🤩
1.別名を「アフリカスミレ」という、アルプムWmES『ルヒスにゴニョと 0:マントポーリア,ヒャクトポボーリア,セントポーリア,ジュウトポーリア
2.別名を「アフリカスミレ」という、アルプムWmES『ルヒスにゴニョと 0
정답 : 2 🤩
1.スイスのフェデラーといえば、どんなスポーツで活躍した男性?:サッカー,テニス,ゴルフ,ボクシング
2.「toto」といえば、どんなスポーツに関するくじ ?
➡️Vectorization :  ['テニス', '野球', 'サッカー', 'ボクシング', 'ゴルフ']
정답 : 0 🤩


ValueError: ' アニメゲーム' is not in list

In [ ]:


# 입력 데이터
texts = ['おにごっこ', 'かくれんぼ', 'かけっこ', 'さかあがり']

# 입력 단어
input_text = 'かくれんばぼ'

# 레벤슈타인 거리 계산
distances = [levenshtein_distance(input_text, text) for text in texts]

# 가장 유사한 단어 추출
most_similar_index = distances.index(min(distances))
most_similar_word = texts[most_similar_index]

print(f"'{input_text}'와 가장 비슷한 단어는 '{most_similar_word}'입니다.")

# 최소 편집 거리를 계산하는 레벤슈타인 알고리즘 사용

'かくれんばぼ'와 가장 비슷한 단어는 'かくれんぼ'입니다.
